In [1]:
!pip install pyspark
!pip install pyarrow
!pip install ydata_profiling
!pip install findspark
!pip install numpy --upgrade

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 317.0 MB 5.9 kB/s  eta 0:00:011
     |████████████████████████████████| 200 kB 68.8 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488513 sha256=5735b328f9f00fb13d2d852dc020c1c7be37228edacc1a1baa28a5ae7289ec8d
  Stored in directory: /home/ubuntu/.cache/pip/wheels/da/78/6d/54350e0243f65f77dccf6ebe2ed5559faf6900559e904fb957
Successfully built pyspark
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 359 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 409 kB 24.9 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 57.4 MB/s eta 0:00:01
     |████████████████████████████████| 296 kB 63.2 MB/s eta 0:00:01
     |████████████████████████████████| 686 kB 49

In [1]:
import findspark
findspark.init() 

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set("spark.sql.execution.arrow.enabled", True)

In [5]:
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "storage.yandexcloud.net")
spark._jsc.hadoopConfiguration().set("fs.s3a.signing-algorithm", "")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key","")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key","")

ОСНОВНЫЕ ПРОБЛЕМЫ:
1. Дупликаты -> сделать дроп
2. Отрицательные значения в колонке customer_id -> сделать фильтрацию по range для определенных колонок
3. Null значения в terminal_id -> заменить null на что-то?

In [13]:
import boto3
session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id = "",
    aws_secret_access_key = ""
    
    
)

In [14]:
file_names = s3.list_objects_v2(Bucket = 'bucket-mlops-fraud-system')

In [22]:
files = [f['Key'] for f in file_names['Contents'] if f['Key'].endswith('.txt')]

In [23]:
files

['2019-08-22.txt',
 '2019-09-21.txt',
 '2019-10-21.txt',
 '2019-11-20.txt',
 '2019-12-20.txt',
 '2020-01-19.txt',
 '2020-02-18.txt',
 '2020-03-19.txt',
 '2020-04-18.txt',
 '2020-05-18.txt',
 '2020-06-17.txt',
 '2020-07-17.txt',
 '2020-08-16.txt',
 '2020-09-15.txt',
 '2020-10-15.txt',
 '2020-11-14.txt',
 '2020-12-14.txt',
 '2021-01-13.txt',
 '2021-02-12.txt',
 '2021-03-14.txt',
 '2021-04-13.txt',
 '2021-05-13.txt',
 '2021-06-12.txt',
 '2021-07-12.txt',
 '2021-08-11.txt',
 '2021-09-10.txt',
 '2021-10-10.txt',
 '2021-11-09.txt',
 '2021-12-09.txt',
 '2022-01-08.txt',
 '2022-02-07.txt',
 '2022-03-09.txt',
 '2022-04-08.txt',
 '2022-05-08.txt',
 '2022-06-07.txt',
 '2022-07-07.txt',
 '2022-08-06.txt',
 '2022-09-05.txt',
 '2022-10-05.txt',
 '2022-11-04.txt']

In [46]:
for f in files:
    print("File name:", f)
    df = spark.read.option('comment', '#').\
                    option('timestampFormat', 'yyyy-MM-dd HH:mm:ss').\
                    schema("tranaction_id LONG, tx_datetime STRING, customer_id INT, terminal_id INT, tx_amount DOUBLE, tx_time_seconds LONG, tx_time_days LONG, tx_fraud INT, tx_fraud_scenario INT").\
                    format('csv').\
                    load(f's3a://bucket-mlops-fraud-system/{f}')
    
    duplicate_rows = df.count() - df.dropDuplicates().count()
    print(f"Number of duplicate rows: {duplicate_rows}")
    df = df.dropDuplicates()
    
    print('/n')
    print('Discription')
    df.select("tranaction_id", "customer_id", "terminal_id", "tx_time_days", "tx_fraud", "tx_fraud_scenario").describe().show()
    print('/n')
    df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

File name: 2019-08-22.txt
Number of duplicate rows: 181
/n
Discription
+-------+--------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|       tranaction_id|       customer_id|       terminal_id|      tx_time_days|          tx_fraud|  tx_fraud_scenario|
+-------+--------------------+------------------+------------------+------------------+------------------+-------------------+
|  count|            46988237|          46988237|          46988237|          46988237|          46988237|           46988237|
|   mean|         2.3494118E7|  500433.721007579|26597.332244897803|14.500636361394022|0.0537792043570394|0.10841483582369775|
| stddev|1.3564335784685532E7|288539.17672027927|1528140.6043842095|  8.65541716356777|0.2255814766746961|0.45687752307129553|
|    min|                   0|           -999999|                 0|                 0|                 0|                  0|
|    max|            46988236|          